In [1]:
%env PGDATABASE=postgres
%env PGHOST=your_IP_addr
%env PGPORT=5432
%env PGUSER=postgres
%env PGPASSWORD=your_password
%env PGINSTANCE=your_instance

env: PGDATABASE=postgres
env: PGHOST=10.66.16.3
env: PGPORT=5432
env: PGUSER=postgres
env: PGPASSWORD=cs327e!
env: PGINSTANCE=pg


### Download Dataset

In [2]:
!gsutil cp gs://cs327e-open-access/postgres_college.zip .

Copying gs://cs327e-open-access/postgres_college.zip...
/ [1 files][  1.9 KiB/  1.9 KiB]                                                
Operation completed over 1 objects/1.9 KiB.                                      


In [3]:
!unzip postgres_college.zip

Archive:  postgres_college.zip
  inflating: class.csv               
  inflating: create_tables.sql       
  inflating: instructor.csv          
  inflating: student.csv             
  inflating: teaches.csv             


### Create tables

In [2]:
!psql -c "\i /home/jupyter/snippets/create_tables.sql"

psql:/home/jupyter/snippets/create_tables.sql:1: NOTICE:  drop cascades to 5 other objects
DETAIL:  drop cascades to table college.student
drop cascades to table college.instructor
drop cascades to table college.teaches
drop cascades to table college.takes
drop cascades to table college.class
DROP SCHEMA
CREATE SCHEMA
SET
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE


In [3]:
!psql -c "\d college.Class"

                       Table "college.class"
 Column  |          Type          | Collation | Nullable | Default 
---------+------------------------+-----------+----------+---------
 sid     | character(12)          |           |          | 
 cno     | character(6)           |           |          | 
 cname   | character varying(200) |           |          | 
 credits | integer                |           |          | 
 grade   | character(2)           |           |          | 



In [4]:
!psql -c "\d college.Student"

                     Table "college.student"
 Column |          Type          | Collation | Nullable | Default 
--------+------------------------+-----------+----------+---------
 sid    | character(12)          |           |          | 
 fname  | character varying(200) |           |          | 
 lname  | character varying(200) |           |          | 
 dob    | date                   |           |          | 
 status | character(3)           |           |          | 



In [5]:
!psql -c "\d college.Instructor"

                    Table "college.instructor"
 Column |          Type          | Collation | Nullable | Default 
--------+------------------------+-----------+----------+---------
 tid    | character(12)          |           |          | 
 name   | character varying(200) |           |          | 
 dept   | character varying(200) |           |          | 



In [6]:
!psql -c "\d college.Teaches"

                 Table "college.teaches"
 Column |     Type      | Collation | Nullable | Default 
--------+---------------+-----------+----------+---------
 tid    | character(12) |           |          | 
 cno    | character(6)  |           |          | 



### Populate Database

In [7]:
!gcloud sql import csv $PGINSTANCE gs://cs327e-open-access/postgres_college/class.csv --database=postgres --table=college.Class --quiet

Importing data into Cloud SQL instance...done.                                 
Imported data from [gs://cs327e-open-access/postgres_college/class.csv] into [https://sqladmin.googleapis.com/sql/v1beta4/projects/cs327e-sp2021/instances/pg].


In [8]:
!gcloud sql import csv $PGINSTANCE gs://cs327e-open-access/postgres_college/student.csv --database=postgres --table=college.Student --quiet

Importing data into Cloud SQL instance...done.                                 
Imported data from [gs://cs327e-open-access/postgres_college/student.csv] into [https://sqladmin.googleapis.com/sql/v1beta4/projects/cs327e-sp2021/instances/pg].


In [9]:
!gcloud sql import csv $PGINSTANCE gs://cs327e-open-access/postgres_college/instructor.csv --database=postgres --table=college.Instructor --quiet

Importing data into Cloud SQL instance...done.                                 
Imported data from [gs://cs327e-open-access/postgres_college/instructor.csv] into [https://sqladmin.googleapis.com/sql/v1beta4/projects/cs327e-sp2021/instances/pg].


In [10]:
!gcloud sql import csv $PGINSTANCE gs://cs327e-open-access/postgres_college/teaches.csv --database=postgres --table=college.Teaches --quiet

Importing data into Cloud SQL instance...done.                                 
Imported data from [gs://cs327e-open-access/postgres_college/teaches.csv] into [https://sqladmin.googleapis.com/sql/v1beta4/projects/cs327e-sp2021/instances/pg].


### Verify data load

In [11]:
!psql -c "select count(*) from college.Class"

 count 
-------
    16
(1 row)



In [12]:
!psql -c "select count(*) from college.Student"

 count 
-------
    12
(1 row)



In [13]:
!psql -c "select count(*) from college.Instructor"

 count 
-------
    10
(1 row)



In [14]:
!psql -c "select count(*) from college.Teaches"

 count 
-------
    12
(1 row)



### Remodel Class table using CTAS statement

In [16]:
!psql -c "create table college.Class2 as select distinct cno, cname, credits from college.Class"

SELECT 12


In [17]:
!psql -c "select * from college.Class2 order by cno"

  cno   |                 cname                 | credits 
--------+---------------------------------------+---------
 CS303E | Elements of Computers and Programming |       3
 CS313E | Elements of Software Design           |       3
 CS313E | Elements of Software Engineering      |       3
 CS326E | Elements of Networking                |       3
 CS327E | Elements of Databases                 |       3
 CS329E | Elements of Web Programming           |       3
 CS331E | Elements of Software Engineering      |       3
 CS347  | Data Management                       |       3
 CS373  | Software Engineering                  |       3
 M328K  | Intro to Number Theory                |       3
 M358K  | Applied Statistics                    |       3
 M362K  | Probability I                         |       3
(12 rows)



In [18]:
!psql -c "delete from college.Class2 where cno = 'CS313E' and cname = 'Elements of Software Engineering'"

DELETE 1


In [19]:
!psql -c "select * from college.Class2 order by cno"

  cno   |                 cname                 | credits 
--------+---------------------------------------+---------
 CS303E | Elements of Computers and Programming |       3
 CS313E | Elements of Software Design           |       3
 CS326E | Elements of Networking                |       3
 CS327E | Elements of Databases                 |       3
 CS329E | Elements of Web Programming           |       3
 CS331E | Elements of Software Engineering      |       3
 CS347  | Data Management                       |       3
 CS373  | Software Engineering                  |       3
 M328K  | Intro to Number Theory                |       3
 M358K  | Applied Statistics                    |       3
 M362K  | Probability I                         |       3
(11 rows)



### Create Takes table from old Class

In [20]:
!psql -c "create table college.Takes as select sid, cno, grade from college.Class"

SELECT 16


In [21]:
!psql -c "select * from college.Takes order by sid"

     sid      |  cno   | grade 
--------------+--------+-------
 aprilz       | CS373  | 
 aprilz       | M358K  | 
 bzen26       | CS313E | B+
 jc           | CS327E | 
 jc           | CS331E | 
 jc           | CS313E | 
 jerryh       | CS329E | A-
 jerryh       | CS327E | B 
 kev18        | CS329E | 
 kev18        | CS347  | 
 kev18        | M362K  | 
 paulg        | CS313E | 
 paulg        | CS329E | A 
 paulg        | CS326E | A-
 sudeepa4     | M328K  | 
 sudeepa4     | CS303E | 
(16 rows)



### Drop and rename Class tables

In [22]:
!psql -c "drop table college.Class"

DROP TABLE


In [23]:
!psql -c "alter table college.Class2 rename to Class"

ALTER TABLE


### Add primary keys

In [24]:
!psql -c "alter table college.Class add primary key (cno)"

ALTER TABLE


In [25]:
!psql -c "alter table college.Student add primary key (sid)"

ALTER TABLE


In [26]:
!psql -c "alter table college.Takes add primary key (sid, cno)"

ALTER TABLE


In [27]:
!psql -c "alter table college.Instructor add primary key (tid)"

ALTER TABLE


In [28]:
!psql -c "alter table college.Teaches add primary key (tid, cno)"

ALTER TABLE


### Add foreign keys

In [29]:
!psql -c "alter table college.Takes add foreign key (sid) references college.Student (sid)"

ALTER TABLE


In [30]:
!psql -c "alter table college.Takes add foreign key (cno) references college.Class (cno)"

ALTER TABLE


In [31]:
!psql -c "alter table college.Teaches add foreign key (tid) references college.Instructor (tid)"

ALTER TABLE


In [32]:
!psql -c "alter table college.Teaches add foreign key (cno) references college.Class (cno)"

ALTER TABLE


### Describe final tables

In [33]:
!psql -c "\d college.Student"

                     Table "college.student"
 Column |          Type          | Collation | Nullable | Default 
--------+------------------------+-----------+----------+---------
 sid    | character(12)          |           | not null | 
 fname  | character varying(200) |           |          | 
 lname  | character varying(200) |           |          | 
 dob    | date                   |           |          | 
 status | character(3)           |           |          | 
Indexes:
    "student_pkey" PRIMARY KEY, btree (sid)
Referenced by:
    TABLE "college.takes" CONSTRAINT "takes_sid_fkey" FOREIGN KEY (sid) REFERENCES college.student(sid)



In [34]:
!psql -c "\d college.Class"

                       Table "college.class"
 Column  |          Type          | Collation | Nullable | Default 
---------+------------------------+-----------+----------+---------
 cno     | character(6)           |           | not null | 
 cname   | character varying(200) |           |          | 
 credits | integer                |           |          | 
Indexes:
    "class_pkey" PRIMARY KEY, btree (cno)
Referenced by:
    TABLE "college.takes" CONSTRAINT "takes_cno_fkey" FOREIGN KEY (cno) REFERENCES college.class(cno)
    TABLE "college.teaches" CONSTRAINT "teaches_cno_fkey" FOREIGN KEY (cno) REFERENCES college.class(cno)



In [35]:
!psql -c "\d college.Takes"

                  Table "college.takes"
 Column |     Type      | Collation | Nullable | Default 
--------+---------------+-----------+----------+---------
 sid    | character(12) |           | not null | 
 cno    | character(6)  |           | not null | 
 grade  | character(2)  |           |          | 
Indexes:
    "takes_pkey" PRIMARY KEY, btree (sid, cno)
Foreign-key constraints:
    "takes_cno_fkey" FOREIGN KEY (cno) REFERENCES college.class(cno)
    "takes_sid_fkey" FOREIGN KEY (sid) REFERENCES college.student(sid)



In [36]:
!psql -c "\d college.Instructor"

                    Table "college.instructor"
 Column |          Type          | Collation | Nullable | Default 
--------+------------------------+-----------+----------+---------
 tid    | character(12)          |           | not null | 
 name   | character varying(200) |           |          | 
 dept   | character varying(200) |           |          | 
Indexes:
    "instructor_pkey" PRIMARY KEY, btree (tid)
Referenced by:
    TABLE "college.teaches" CONSTRAINT "teaches_tid_fkey" FOREIGN KEY (tid) REFERENCES college.instructor(tid)



In [37]:
!psql -c "\d college.Teaches"

                 Table "college.teaches"
 Column |     Type      | Collation | Nullable | Default 
--------+---------------+-----------+----------+---------
 tid    | character(12) |           | not null | 
 cno    | character(6)  |           | not null | 
Indexes:
    "teaches_pkey" PRIMARY KEY, btree (tid, cno)
Foreign-key constraints:
    "teaches_cno_fkey" FOREIGN KEY (cno) REFERENCES college.class(cno)
    "teaches_tid_fkey" FOREIGN KEY (tid) REFERENCES college.instructor(tid)

